In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
from lxml import objectify
from lxml import etree
import xml.dom.minidom

import sys
sys.path.append(r'/home/jamesrowland/Documents/Code/blimp')
sys.path.append(r'/home/jamesrowland/Documents/Code/blimp/sdk')

from utils import parse_markpoints
from prairie_interface import prairie_interface

#from SLM_sdk import SLM_sdk

In [60]:
class b_52(parse_markpoints, prairie_interface):
    
    ''' class to automatically run the naparam experiment in naparm_path '''
    
    def __init__(self, naparm_path, naparm_hz=1000):
        
        self.naparm_path = naparm_path
        self.naparm_hz = naparm_hz
        self.get_paths()
        self.get_masks()
        self.trigger_times(self.trigger_path)
        
        self.n_trials = self.num_stims / self.num_masks
        
        #check that the number of trials logic is correct
        assert self.n_trials.is_integer() == True and self.n_trials > 0
        self.n_trials = int(self.n_trials)
        
        #init the SLM sdk class
        #self.sdk = SLM_sdk()
        #self.sdk.SLM_connect()
        
        #inherits the attributes of classes parse_markpoints and prairie_interface
        super(b_52, self).__init__(self.xml_path, self.gpl_path)
        
        
        self.markpoints_strings = self.build_strings()
          
    def get_paths(self):
        
        for file in os.listdir(naparm_path):
            
            file_path = os.path.join(naparm_path, file)

            if file == 'ToSLM_AllTrials.dat':
                self.trigger_path = file_path
            elif file.endswith('.gpl'):
                self.gpl_path = file_path
            elif file.endswith('.xml'):
                self.xml_path = file_path
            elif file == 'PhaseMasks':
                self.mask_path = file_path
                
    def get_masks(self):
        
        self.mask_files = []

        for file in os.listdir(self.mask_path):
            file_path = os.path.join(self.mask_path, file)
            
            if file.endswith('.tif') or file.endswith('.tiff'):
                self.mask_files.append(file_path)
        
        # i think this puts the files in the order expected by naparm, but this is a potential error source
        self.mask_files.sort()
        
        self.num_masks = len(self.mask_files)
                
        
    def trigger_times(self, dat_path):
    
        ''' 
        function to detect edges signalling trigger onset in dat files
        input: dat file path
        output: times of trigger onsets in dat file (seconds)

        '''

        arr = np.fromfile(dat_path, dtype=float)

        #edge detect the SLM trigger dat file      
        edges = [i for i,j in enumerate(arr) if arr[i-1] == 0 and arr[i] == 5]

        # above list doesnt hold for triggers sent at sample 0
        if arr[0] == 5:
            edges[0] = 0

        self.triggers = np.array(edges) / self.naparm_hz
        self.num_stims = len(self.triggers)

        self.sleep_times = np.diff(self.triggers)


    
    def drop_naparm(self):
        
        ''' runs the experiment outlined in the naparm folder
            currently only works when the first shot is at time 0
        '''
        
        #repeat the masks paths n_trials times. So can be iterated through on each trial
        rep_masks = np.tile(self.mask_files, self.n_trials)
        
        # get the first mask and pop it out, so it is not iterated through
        # this may not work due to needing to load mp
        mask1 = rep_masks[0]
        rep_masks = np.delete(rep_masks, 0)
        
        #load the first mask
        #sdk.load_mask(mask1)  
        time.sleep(1)
        
        for trial,mask in enumerate(rep_masks):
            
            #fire markpoints
            #this doesnt work yet due to indexing
            pl.SendScriptCommands(markpoints_strings[trial])
            
            #sdk.load_mask(mask)
            print('load mask')
                
            time.sleep(self.sleep_times[trial])
        
        
    

       
        

In [62]:
t1 = time.time()

naparm_path = r'/home/jamesrowland/Desktop/2019-01-16_plasticSlide_NAPARM_001'
anp = b_52(naparm_path)
anp.drop_naparm()

t2 = time.time()

t2 - t1

failed to connect to PV
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask
load mask


30.05355453491211

In [45]:
m = mf.pop()

AttributeError: 'numpy.ndarray' object has no attribute 'pop'

In [44]:
len(anp.sleep_times)

29